In [7]:
import os
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium.spaces import Box, MultiDiscrete
from stable_baselines3 import PPO
import matplotlib.pyplot as plt
import random

# Function to load and preprocess data
def load_and_preprocess_data(house_dir):
    # Load electric data
    electric_dir = os.path.join(house_dir, 'Electric_data')
    electric_csv_files = [f for f in os.listdir(electric_dir) if f.endswith('.csv') and f != 'appliances_metadata.csv']
    electric_dfs = []
    for csv_file in electric_csv_files:
        file_path = os.path.join(electric_dir, csv_file)
        df = pd.read_csv(file_path)
        # Parse timestamp
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)
        electric_dfs.append(df)
    # Concatenate all months
    electric_df = pd.concat(electric_dfs)
    # Resample to 1-minute intervals
    electric_df = electric_df.resample('1T').mean()
    # Forward-fill small gaps (up to 2 missing intervals)
    electric_df = electric_df.ffill(limit=2)
    # Handle missing values
    electric_df = electric_df.dropna()

    # Load environmental data
    env_dir = os.path.join(house_dir, 'Environmental_data')
    env_csv_files = [f for f in os.listdir(env_dir) if f.endswith('.csv')]
    env_dfs = []
    for csv_file in env_csv_files:
        file_path = os.path.join(env_dir, csv_file)
        df = pd.read_csv(file_path)
        # Parse timestamp
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)
        env_dfs.append(df)
    # Concatenate all months
    env_df = pd.concat(env_dfs)
    # Resample to 1-minute intervals
    env_df = env_df.resample('1T').interpolate()
    # Forward-fill small gaps (up to 60 minutes)
    env_df = env_df.ffill(limit=60)
    # Handle missing values
    env_df = env_df.dropna()

    # Merge electric and environmental data on timestamp index
    data = electric_df.join(env_df, how='inner')
    # Now data has both electric and environmental data, and timestamps are aligned

    return data

# Function to load per-house appliance metadata
def load_appliances_metadata(house_dir):
    # Load appliances_metadata.csv from the Electric_data directory
    appliances_metadata_file = os.path.join(house_dir, 'Electric_data', 'appliances_metadata.csv')
    appliances_metadata = {}
    if os.path.exists(appliances_metadata_file):
        df = pd.read_csv(appliances_metadata_file)
        # Clean column names (remove spaces and brackets)
        df.columns = [col.strip().replace(' [W]', '').replace(' (sec)', '').replace(' ', '_').lower() for col in df.columns]
        for _, row in df.iterrows():
            appliance_name = row['appliance']
            appliances_metadata[appliance_name] = {
                'wattage': row.get('wattage', np.nan),
                'threshold': row.get('threshold', np.nan),
                'min_on': row.get('min_on', np.nan),
                'min_off': row.get('min_off', np.nan),
                # Include capacity if applicable (e.g., for AC units)
                # For the example, we'll assume capacity is negative wattage for cooling appliances
                'capacity': -row['wattage'] if 'ac' in appliance_name.lower() else row.get('capacity', np.nan)
            }
    else:
        # Handle case where appliances_metadata.csv is missing
        print(f"Warning: appliances_metadata.csv not found in {house_dir}. Using default metadata.")
    return appliances_metadata

# Function to load socio-economic data
def load_socio_economic_data(house_dir):
    # Socio-economic data file
    socio_file = os.path.join(house_dir, 'Sociodemographic', 'socioeconomic_data.csv')
    if os.path.exists(socio_file):
        df = pd.read_csv(socio_file)
        # Convert to dictionary
        socio_data = df.to_dict(orient='records')[0]  # Assuming one record per house
    else:
        print(f"Warning: socioeconomic_data.csv not found in {house_dir}. Using default socio-economic data.")
        socio_data = {}
    return socio_data

# Function to generate personalized preferences
def generate_personalized_preferences(socio_data, appliances):
    preferences = {}
    # Generate preferred temperature range based on age
    age = socio_data.get('Age of the respondent', 35)
    if age >= 65:
        # Older individuals may prefer warmer temperatures
        preferences['temperature_range'] = [22.0, 27.0]
    elif age <= 25:
        # Younger individuals may prefer cooler temperatures
        preferences['temperature_range'] = [18.0, 23.0]
    else:
        # Default temperature range
        preferences['temperature_range'] = [20.0, 25.0]

    # Generate device usage preferences
    preferences['device_usage'] = {}
    for appliance in appliances:
        # Skip appliances that are always on (e.g., fridge)
        if appliance.startswith('fridge'):
            continue

        # Randomly select preferred days and hours for usage
        preferred_days = random.sample(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], k=random.randint(2, 7))
        preferred_hours = random.sample(range(0, 24), k=random.randint(2, 6))

        # Appliance-specific adjustments
        if appliance == 'washing_machine':
            # Likely used on weekends
            preferred_days = ['Saturday', 'Sunday']
            preferred_hours = random.sample(range(8, 20), k=2)  # Daytime hours
        elif appliance == 'kettle':
            # Likely used in the morning and evening
            preferred_hours = [7, 8, 9, 18, 19, 20]
            preferred_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
        elif appliance.startswith('ac'):
            # AC usage depends on external temperature; we'll assume preferred temperature range handles this
            continue
        elif appliance == 'boiler':
            # Likely used in the morning and evening
            preferred_hours = [6, 7, 8, 21, 22, 23]
            preferred_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        elif appliance == 'dish_washer':
            # Likely used after dinner
            preferred_hours = [20, 21, 22]
            preferred_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

        # Add to preferences
        preferences['device_usage'][appliance] = {
            'days': preferred_days,
            'hours': preferred_hours
        }

    return preferences

# Local Home Energy Management System Environment
class LHEMSEnv(gym.Env):
    metadata = {
        "render_modes": ["human"],
    }

    def __init__(self, data, appliances_metadata, comfort_prefs, socio_data, global_appliance_list, preferences):
        super().__init__()
        self.data = data
        self.appliances_metadata = appliances_metadata
        self.socio_data = socio_data  # Socio-economic data
        self.global_appliance_list = global_appliance_list  # Global list of all appliances
        self.preferences = preferences  # Personalized preferences

        # Determine the appliances present in this house
        self.house_appliance_names = [col for col in data.columns if col in global_appliance_list]
        self.num_appliances = len(self.global_appliance_list)
        self.current_time_index = 0
        self.max_time_steps = len(data)

        # Adjust comfort preferences based on socio-economic data and personalized preferences
        self.comfort_prefs = self.adjust_comfort_prefs(comfort_prefs)

        # Create a mask for appliances not present in the house
        self.appliance_mask = np.array([1 if appliance in self.house_appliance_names else 0 for appliance in self.global_appliance_list])

        # Define action space: Each appliance can be ON or OFF
        self.action_space = gym.spaces.MultiDiscrete([2]*self.num_appliances)

        # Define observation space
        # For each appliance: [Normalized Power, Desired State]
        # Environmental data: [Internal Temp, Internal Humidity, External Temp, External Humidity]
        # Time Features: [Hour, Day of Week]
        obs_low = [0.0]*(2*self.num_appliances) + [-50.0]*4 + [0.0, 0.0]
        obs_high = [1.0]*(2*self.num_appliances) + [50.0]*4 + [1.0, 1.0]
        self.observation_space = gym.spaces.Box(
            low=np.array(obs_low, dtype=np.float32),
            high=np.array(obs_high, dtype=np.float32),
            dtype=np.float32
        )

        self.R = self.socio_data.get('Thermal Resistance', 0.05)  # Adjusted value
        self.C = self.socio_data.get('Thermal Capacitance', 1e6)  # Adjusted value
        self.delta_t = 60  # Time step in seconds

        # # Thermal model parameters
        # self.R = self.socio_data.get('Thermal Resistance', 2.0)  # Thermal resistance
        # self.C = self.socio_data.get('Thermal Capacitance', 500.0)  # Thermal capacitance
        # self.internal_temperature = data['internal_temperature'].iloc[0]  # Initial temperature

        # # HVAC parameters
        # self.delta_t = 60  # Time step in seconds (1 minute)

    def adjust_comfort_prefs(self, base_prefs):
        adjusted_prefs = base_prefs.copy()

        # Use personalized temperature range
        adjusted_prefs['temperature_range'] = self.preferences.get('temperature_range', base_prefs.get('temperature_range', [20.0, 25.0]))

        # Adjust cost sensitivity based on income
        income_bracket = self.socio_data.get('Family monthly income', 'Medium')
        if income_bracket == 'High':
            # Less sensitive to energy cost
            adjusted_prefs['cost_weight'] = 0.5
            adjusted_prefs['comfort_weight'] = 1.5
        elif income_bracket == 'Low':
            # More sensitive to energy cost
            adjusted_prefs['cost_weight'] = 1.5
            adjusted_prefs['comfort_weight'] = 0.5
        else:
            # Default weights
            adjusted_prefs['cost_weight'] = 1.0
            adjusted_prefs['comfort_weight'] = 1.0

        # Adjust comfort preferences if pets are present
        pets = self.socio_data.get('Pets', 'No pets')
        if pets != 'No pets':
            # Increase comfort weight
            adjusted_prefs['comfort_weight'] += 0.5

        return adjusted_prefs

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_time_index = 0
        self.done = False
        self.internal_temperature = self.data['internal_temperature'].iloc[0]  # Reset internal temperature
        observation = self._get_obs()
        info = {}
        return observation, info

    def step(self, action):
        self._take_action(action)
        self._update_environment()
        observation = self._get_obs()
        reward = self._calculate_reward(action)
        terminated = self._is_terminated()
        truncated = False
        info = {}

        self.current_time_index += 1

        print(f"Time Step: {self.current_time_index}, Internal Temp: {self.internal_temperature:.2f} °C")

        return observation, reward, terminated, truncated, info

    def _update_environment(self):
        # Update internal temperature based on the thermal model
        external_temp = self.data['external_temperature'].iloc[self.current_time_index]
        delta_t = self.delta_t  # Time step in seconds (1 minute)

        # Determine HVAC power based on action
        P_HVAC = 0
        for idx, appliance_name in enumerate(self.global_appliance_list):
            if self.appliance_mask[idx] == 1 and 'ac' in appliance_name.lower():
                hvac_action = self.actions[idx]
                # Get capacity from per-house appliances_metadata
                ac_metadata = self.appliances_metadata.get(appliance_name, {})
                ac_capacity = ac_metadata.get('capacity', -5000.0)  # Default capacity if not specified
                P_HVAC += ac_capacity * hvac_action  # Sum the capacities of all active AC units

        # Calculate internal heat gains (simplified)
        Q_int = self.calculate_internal_heat_gains()

        # Update internal temperature
        dT = delta_t / self.C * (-1 / self.R * (self.internal_temperature - external_temp) + P_HVAC + Q_int)
        self.internal_temperature += dT

    def calculate_internal_heat_gains(self):
        # Simplified calculation
        num_occupants = self.socio_data.get('Number of Occupants', 1)
        heat_per_person = 100.0  # Watts per person
        return num_occupants * heat_per_person

    def _get_obs(self):
        obs = []
        timestamp = self.data.index[self.current_time_index]

        # Appliance observations
        for appliance_name in self.global_appliance_list:
            if appliance_name in self.house_appliance_names:
                power = self.data[appliance_name].iloc[self.current_time_index]
                # Normalize power based on appliance metadata
                appliance_metadata = self.appliances_metadata.get(appliance_name, {})
                max_power = appliance_metadata.get('wattage', 1.0)
                normalized_power = power / max_power if max_power > 0 else 0.0
                desired_state = self.get_desired_state(appliance_name, timestamp)
            else:
                # For appliances not present, set observations to zeros
                normalized_power = 0.0
                desired_state = 0
            obs.extend([normalized_power, desired_state])

        # Environmental data
        internal_temp = self.internal_temperature  # Updated internal temperature
        internal_humidity = self.data['internal_humidity'].iloc[self.current_time_index]
        external_temp = self.data['external_temperature'].iloc[self.current_time_index]
        external_humidity = self.data['external_humidity'].iloc[self.current_time_index]
        obs.extend([internal_temp, internal_humidity, external_temp, external_humidity])

        # Time features
        hour = timestamp.hour / 23.0  # Normalize to [0,1]
        day_of_week = timestamp.dayofweek / 6.0  # Normalize to [0,1]
        obs.extend([hour, day_of_week])

        return np.array(obs, dtype=np.float32)

    def get_desired_state(self, appliance_name, timestamp):
        # Determine desired state based on usage preferences
        day_name = timestamp.day_name()
        hour = timestamp.hour
        device_prefs = self.preferences.get('device_usage', {}).get(appliance_name, {})
        preferred_days = device_prefs.get('days', [])
        preferred_hours = device_prefs.get('hours', [])
        if (preferred_days and day_name in preferred_days) and (preferred_hours and hour in preferred_hours):
            return 1  # Desired to be ON
        else:
            return 0  # Desired to be OFF

    def _take_action(self, action):
        # Apply action mask to ignore actions for appliances not present
        self.actions = action * self.appliance_mask

    def _calculate_reward(self, action):
        total_reward = 0
        timestamp = self.data.index[self.current_time_index]

        # Environmental comfort parameters
        internal_temp = self.internal_temperature
        desired_temp_range = self.comfort_prefs.get('temperature_range', [20.0, 25.0])
        temp_penalty = self.comfort_prefs.get('temp_penalty', 5.0)

        # Cost and comfort weights
        cost_weight = self.comfort_prefs.get('cost_weight', 1.0)
        comfort_weight = self.comfort_prefs.get('comfort_weight', 1.0)

        for idx, appliance_name in enumerate(self.global_appliance_list):
            if self.appliance_mask[idx] == 0:
                continue  # Skip appliances not present in the house

            appliance_metadata = self.appliances_metadata.get(appliance_name, {})
            max_power = appliance_metadata.get('wattage', 1.0)
            actual_state = action[idx]
            price = 1.0  # Modify as needed

            # Negative electric cost
            energy_cost = -cost_weight * price * max_power * actual_state

            # Comfort penalty
            comfort_penalty = 0

            # Appliance-specific comfort preferences
            desired_state = self.get_desired_state(appliance_name, timestamp)
            appliance_prefs = self.comfort_prefs.get(appliance_name, {})
            if desired_state == 1 and actual_state == 0:
                comfort_penalty += comfort_weight * appliance_prefs.get('penalty', 10.0)

            # For AC units, consider temperature comfort
            if 'ac' in appliance_name.lower() and actual_state == 0:
                if internal_temp > desired_temp_range[1]:
                    comfort_penalty += comfort_weight * temp_penalty * (internal_temp - desired_temp_range[1])
                elif internal_temp < desired_temp_range[0]:
                    comfort_penalty += comfort_weight * temp_penalty * (desired_temp_range[0] - internal_temp)

            # Total reward for the appliance
            reward = energy_cost - comfort_penalty
            total_reward += reward

        return total_reward

    def _is_terminated(self):
        return self.current_time_index >= self.max_time_steps - 1

    def render(self):
        pass

    def close(self):
        pass

# Function to train local models for each LHEMS
def train_local_model(env, global_params=None, total_timesteps=10000):
    # Initialize the local model
    model = PPO('MlpPolicy', env, verbose=0)
    if global_params:
        model.policy.load_state_dict(global_params)
    # Train the model
    model.learn(total_timesteps=total_timesteps)
    # Return the trained model and its parameters
    return model, model.policy.state_dict()

# Function to aggregate global model parameters
def aggregate_global_model(local_models_params):
    global_params = {}
    num_models = len(local_models_params)
    for key in local_models_params[0].keys():
        # Average the parameters across all local models
        global_params[key] = sum([model_params[key] for model_params in local_models_params]) / num_models
    return global_params

# Evaluation function for both RL agent and baseline
def evaluate_policy(env, policy_func, num_episodes=1):
    """
    Evaluates a given policy on the environment.

    Parameters:
    - env: The environment to evaluate on.
    - policy_func: A function that takes the environment and returns an action.
    - num_episodes: Number of episodes to run for evaluation.

    Returns:
    - metrics: Dictionary containing evaluation metrics.
    """
    metrics = {
        'total_energy_consumption': 0.0,
        'total_comfort_violations': 0,
        'total_comfort_violation_severity': 0.0,
        'total_reward': 0.0,
        'energy_cost': 0.0,
        'comfort_penalty': 0.0,
        'action_counts': np.zeros(len(env.global_appliance_list)),
        'time_steps': 0,
        'temperature_record': [],
        'desired_temperature_range': env.comfort_prefs.get('temperature_range', [20.0, 25.0]),
        'internal_temperatures': [],
        'timestamps': [],
    }

    for episode in range(num_episodes):
        obs, _ = env.reset()
        done = False
        while not done:
            action = policy_func(env)
            obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # Update metrics
            # Adjusted indexing to prevent IndexError
            timestamp = env.data.index[env.current_time_index - 1]
            metrics['timestamps'].append(timestamp)
            metrics['time_steps'] += 1
            metrics['total_reward'] += reward

            # Collect internal temperature
            internal_temp = env.internal_temperature
            metrics['internal_temperatures'].append(internal_temp)

            # Energy consumption and cost
            for idx, appliance_name in enumerate(env.global_appliance_list):
                if env.appliance_mask[idx] == 0:
                    continue  # Skip appliances not present
                appliance_metadata = env.appliances_metadata.get(appliance_name, {})
                max_power = appliance_metadata.get('wattage', 1.0)
                actual_state = env.actions[idx]
                power_consumed = max_power * actual_state  # Assuming full power when ON
                metrics['total_energy_consumption'] += power_consumed
                # Assuming price = 1.0 for simplicity
                price = 1.0
                energy_cost = -env.comfort_prefs.get('cost_weight', 1.0) * price * power_consumed
                metrics['energy_cost'] += energy_cost

            # Comfort penalties
            comfort_penalty = 0.0
            desired_temp_range = metrics['desired_temperature_range']
            if internal_temp > desired_temp_range[1]:
                severity = internal_temp - desired_temp_range[1]
                comfort_penalty += env.comfort_prefs.get('comfort_weight', 1.0) * env.comfort_prefs.get('temp_penalty', 5.0) * severity
                metrics['total_comfort_violations'] += 1
                metrics['total_comfort_violation_severity'] += severity
            elif internal_temp < desired_temp_range[0]:
                severity = desired_temp_range[0] - internal_temp
                comfort_penalty += env.comfort_prefs.get('comfort_weight', 1.0) * env.comfort_prefs.get('temp_penalty', 5.0) * severity
                metrics['total_comfort_violations'] += 1
                metrics['total_comfort_violation_severity'] += severity

            metrics['comfort_penalty'] += -comfort_penalty

            # Action counts
            metrics['action_counts'] += env.actions

    # Average metrics over time steps
    metrics['average_energy_consumption'] = metrics['total_energy_consumption'] / metrics['time_steps']
    metrics['average_reward'] = metrics['total_reward'] / metrics['time_steps']
    metrics['average_energy_cost'] = metrics['energy_cost'] / metrics['time_steps']
    metrics['average_comfort_penalty'] = metrics['comfort_penalty'] / metrics['time_steps']
    metrics['comfort_violation_rate'] = metrics['total_comfort_violations'] / metrics['time_steps']
    metrics['average_comfort_violation_severity'] = metrics['total_comfort_violation_severity'] / metrics['time_steps']

    return metrics

# Baseline policy function
def baseline_policy(env):
    """
    A simple rule-based baseline policy.
    For appliances that should be ON according to desired state, turn them ON.
    For others, turn them OFF.
    """
    desired_states = []
    for idx, appliance_name in enumerate(env.global_appliance_list):
        if env.appliance_mask[idx] == 0:
            desired_states.append(0)
            continue  # Skip appliances not present
        desired_state = env.get_desired_state(appliance_name, env.data.index[env.current_time_index])
        desired_states.append(desired_state)
    action = np.array(desired_states)
    return action

# Visualization functions
def plot_internal_temperature(metrics):
    timestamps = metrics['timestamps']
    internal_temps = metrics['internal_temperatures']
    desired_range = metrics['desired_temperature_range']

    plt.figure(figsize=(12, 6))
    plt.plot(timestamps, internal_temps, label='Internal Temperature')
    plt.axhline(y=desired_range[0], color='r', linestyle='--', label='Desired Temp Range')
    plt.axhline(y=desired_range[1], color='r', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Temperature (°C)')
    plt.title('Internal Temperature Over Time')
    plt.legend()
    plt.show()

def plot_action_distribution(metrics, env):
    action_counts = metrics['action_counts']
    appliance_names = env.global_appliance_list

    plt.figure(figsize=(12, 6))
    plt.bar(appliance_names, action_counts)
    plt.xlabel('Appliances')
    plt.ylabel('Number of Times Turned ON')
    plt.title('Appliance Action Distribution')
    plt.xticks(rotation=45)
    plt.show()

def plot_reward_components(metrics):
    timestamps = metrics['timestamps']
    energy_costs = [metrics['energy_cost']] * len(timestamps)
    comfort_penalties = [metrics['comfort_penalty']] * len(timestamps)

    plt.figure(figsize=(12, 6))
    plt.plot(timestamps, energy_costs, label='Energy Cost')
    plt.plot(timestamps, comfort_penalties, label='Comfort Penalty')
    plt.xlabel('Time')
    plt.ylabel('Cost')
    plt.title('Reward Components Over Time')
    plt.legend()
    plt.show()

# Federated training loop
def federated_training(lhems_envs, num_rounds=5, local_timesteps=10000):
    # Initialize local models
    local_models = [None] * len(lhems_envs)
    global_params = None

    for round in range(num_rounds):
        print(f"--- Federated Training Round {round+1}/{num_rounds} ---")
        local_models_params = []
        # Local training at each LHEMS
        for idx, env in enumerate(lhems_envs):
            print(f"Training local model for LHEMS {idx+1}")
            model, local_params = train_local_model(env, global_params=global_params, total_timesteps=local_timesteps)
            local_models_params.append(local_params)
            local_models[idx] = model
        # Global aggregation at GS
        global_params = aggregate_global_model(local_models_params)
        print("Global model aggregated and updated.")

    # Return the final global model parameters and local models
    return global_params, local_models

# Main execution
if __name__ == "__main__":
    # Directory containing house folders
    data_dir = r"C:\Users\hboki\OneDrive - ku.ac.ae\plegmaDataset_clean\Clean_Dataset"
    house_dirs = [os.path.join(data_dir, d) for d in os.listdir(data_dir) if d.startswith('House')]

    # Base comfort preferences
    base_comfort_prefs = {
        'ac_1': {'penalty': 5.0},
        'ac_2': {'penalty': 5.0},
        'ac_3': {'penalty': 5.0},
        'boiler': {'penalty': 10.0},
        'fridge': {'penalty': 8.0},
        'fridge_1': {'penalty': 8.0},
        'fridge_2': {'penalty': 8.0},
        'dish_washer': {'penalty': 6.0},
        'washing_machine': {'penalty': 6.0},
        'kettle': {'penalty': 4.0},
        'temperature_range': [20.0, 25.0],  # Default desired indoor temperature range
        'temp_penalty': 0.1,  # Penalty per degree outside desired range
        'cost_weight': 1.0,   # Default cost weight
        'comfort_weight': 1.0, # Default comfort weight
    }

    # Initialize global appliance set
    global_appliance_set = set()

    # Create LHEMS environments and models for each house
    lhems_envs = []
    appliances_metadata_list = []  # Keep track of appliances metadata for each house
    for house_dir in house_dirs:
        # Load merged data
        data = load_and_preprocess_data(house_dir)
        # Load per-house appliance metadata
        appliances_metadata = load_appliances_metadata(house_dir)
        appliances_metadata_list.append(appliances_metadata)
        # Load socio-economic data
        socio_data = load_socio_economic_data(house_dir)
        # Determine appliances in the house
        house_appliance_names = [col for col in data.columns if col not in ['V', 'A', 'P_agg', 'issues',
                                                                            'internal_temperature', 'internal_humidity',
                                                                            'external_temperature', 'external_humidity']]
        # Update global appliance set
        global_appliance_set.update(house_appliance_names)
        # Generate personalized preferences
        preferences = generate_personalized_preferences(socio_data, house_appliance_names)
        # Create environment with a placeholder global_appliance_list (will update later)
        env = LHEMSEnv(data, appliances_metadata, base_comfort_prefs, socio_data, [], preferences)
        lhems_envs.append(env)

    # Now that we have the complete global appliance list, we can update environments accordingly
    global_appliance_list = sorted(list(global_appliance_set))  # Sorted to maintain consistent order

    # Update environments with the complete global appliance list
    for idx, env in enumerate(lhems_envs):
        env.global_appliance_list = global_appliance_list
        env.num_appliances = len(global_appliance_list)
        # Re-create action space and appliance mask
        env.action_space = gym.spaces.MultiDiscrete([2]*env.num_appliances)
        env.appliance_mask = np.array([1 if appliance in env.house_appliance_names else 0 for appliance in env.global_appliance_list])
        # Re-create observation space
        obs_low = [0.0]*(2*env.num_appliances) + [-50.0]*4 + [0.0, 0.0]
        obs_high = [1.0]*(2*env.num_appliances) + [50.0]*4 + [1.0, 1.0]
        env.observation_space = gym.spaces.Box(
            low=np.array(obs_low, dtype=np.float32),
            high=np.array(obs_high, dtype=np.float32),
            dtype=np.float32
        )
        # Update appliances_metadata if necessary
        env.appliances_metadata = appliances_metadata_list[idx]
        # Reset the environment to ensure it uses the updated settings
        env.reset()

    # Run federated training
    final_global_params, local_models = federated_training(lhems_envs, num_rounds=5, local_timesteps=10000)
    print("Federated training completed.")


    # Evaluate models and baseline
    evaluation_metrics_rl = []
    evaluation_metrics_baseline = []
    for idx, env in enumerate(lhems_envs):
        print(f"Evaluating models for LHEMS {idx+1}")

        # Initialize model with final global parameters
        model = PPO('MlpPolicy', env, verbose=0)
        model.policy.load_state_dict(final_global_params)

        # Define policy functions
        def rl_policy(env):
            obs = env._get_obs()
            action, _states = model.predict(obs, deterministic=True)
            return action

        # Evaluate the RL agent
        metrics_rl = evaluate_policy(env, rl_policy)
        evaluation_metrics_rl.append(metrics_rl)

        # Evaluate the baseline policy
        metrics_baseline = evaluate_policy(env, baseline_policy)
        evaluation_metrics_baseline.append(metrics_baseline)

        # Print evaluation results for RL agent
        print(f"--- Evaluation Metrics for RL Agent in LHEMS {idx+1} ---")
        print(f"Total Energy Consumption: {metrics_rl['total_energy_consumption']:.2f} Wh")
        print(f"Average Energy Consumption per Time Step: {metrics_rl['average_energy_consumption']:.2f} Wh")
        print(f"Total Comfort Violations: {metrics_rl['total_comfort_violations']}")
        print(f"Comfort Violation Rate: {metrics_rl['comfort_violation_rate']:.2%}")
        print(f"Average Comfort Violation Severity: {metrics_rl['average_comfort_violation_severity']:.2f} °C")
        print(f"Total Reward: {metrics_rl['total_reward']:.2f}")
        print(f"Average Reward per Time Step: {metrics_rl['average_reward']:.2f}")
        print(f"Total Energy Cost: {metrics_rl['energy_cost']:.2f}")
        print(f"Total Comfort Penalty: {metrics_rl['comfort_penalty']:.2f}")
        print(f"Action Counts: {metrics_rl['action_counts']}")
        print()

              # Print evaluation results for Baseline policy
        print(f"--- Evaluation Metrics for Baseline Policy in LHEMS {idx+1} ---")
        print(f"Total Energy Consumption: {metrics_baseline['total_energy_consumption']:.2f} Wh")
        print(f"Average Energy Consumption per Time Step: {metrics_baseline['average_energy_consumption']:.2f} Wh")
        print(f"Total Comfort Violations: {metrics_baseline['total_comfort_violations']}")
        print(f"Comfort Violation Rate: {metrics_baseline['comfort_violation_rate']:.2%}")
        print(f"Average Comfort Violation Severity: {metrics_baseline['average_comfort_violation_severity']:.2f} °C")
        print(f"Total Reward: {metrics_baseline['total_reward']:.2f}")
        print(f"Average Reward per Time Step: {metrics_baseline['average_reward']:.2f}")
        print(f"Total Energy Cost: {metrics_baseline['energy_cost']:.2f}")
        print(f"Total Comfort Penalty: {metrics_baseline['comfort_penalty']:.2f}")
        print(f"Action Counts: {metrics_baseline['action_counts']}")
        print()

        # Plotting (optional)
        print("Plotting results for RL Agent...")
        plot_internal_temperature(metrics_rl)
        plot_action_distribution(metrics_rl, env)
        plot_reward_components(metrics_rl)

        print("Plotting results for Baseline Policy...")
        plot_internal_temperature(metrics_baseline)
        plot_action_distribution(metrics_baseline, env)
        plot_reward_components(metrics_baseline)

    # Save the final global model parameters
    print("Final global model parameters are ready for use.")


C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  electric_df = electric_df.resample('1T').mean()
C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:46: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  env_df = env_df.resample('1T').interpolate()


C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  electric_df = electric_df.resample('1T').mean()
C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:46: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  env_df = env_df.resample('1T').interpolate()


C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  electric_df = electric_df.resample('1T').mean()
C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:46: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  env_df = env_df.resample('1T').interpolate()


C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  electric_df = electric_df.resample('1T').mean()
C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:46: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  env_df = env_df.resample('1T').interpolate()


C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  electric_df = electric_df.resample('1T').mean()
C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:46: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  env_df = env_df.resample('1T').interpolate()


...

C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  electric_df = electric_df.resample('1T').mean()
C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:46: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  env_df = env_df.resample('1T').interpolate()


C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  electric_df = electric_df.resample('1T').mean()
C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:46: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  env_df = env_df.resample('1T').interpolate()


C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  electric_df = electric_df.resample('1T').mean()
C:\Users\hboki\AppData\Local\Temp\ipykernel_25628\1148510240.py:46: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  env_df = env_df.resample('1T').interpolate()


--- Federated Training Round 1/5 ---
Training local model for LHEMS 1
Time Step: 1, Internal Temp: 21.60 °C
Time Step: 2, Internal Temp: 21.59 °C
Time Step: 3, Internal Temp: 21.59 °C
Time Step: 4, Internal Temp: 21.58 °C
Time Step: 5, Internal Temp: 21.58 °C
Time Step: 6, Internal Temp: 21.57 °C
Time Step: 7, Internal Temp: 21.57 °C
Time Step: 8, Internal Temp: 21.57 °C
Time Step: 9, Internal Temp: 21.56 °C
Time Step: 10, Internal Temp: 21.56 °C
Time Step: 11, Internal Temp: 21.55 °C
Time Step: 12, Internal Temp: 21.55 °C
Time Step: 13, Internal Temp: 21.54 °C
Time Step: 14, Internal Temp: 21.54 °C
Time Step: 15, Internal Temp: 21.53 °C
Time Step: 16, Internal Temp: 21.53 °C
Time Step: 17, Internal Temp: 21.53 °C
Time Step: 18, Internal Temp: 21.52 °C
Time Step: 19, Internal Temp: 21.52 °C
Time Step: 20, Internal Temp: 21.51 °C
Time Step: 21, Internal Temp: 21.51 °C
Time Step: 22, Internal Temp: 21.50 °C
Time Step: 23, Internal Temp: 21.50 °C
Time Step: 24, Internal Temp: 21.50 °C
Tim

KeyboardInterrupt: 